In [1]:
import sys

import timeit
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

sys.path.append('/home/hoheon/packages/')

from utils.keras import get_model_memory_usage
from utils.numpy import to_onehot

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

%load_ext autoreload
%autoreload 2

Num GPUs Available:  2


In [2]:
# Given data
train, test = tf.keras.datasets.cifar10.load_data()
tf.config.list_physical_devices()
x_train, y_train = train[0], train[1]
x_test, y_test = test[0], test[1]

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

y_train = to_onehot(y_train, 10).reshape(-1, 10)
y_val = to_onehot(y_val, 10).reshape(-1, 10)
y_test = to_onehot(y_test, 10).reshape(-1, 10)

print(x_train.shape, y_train.shape)
print("N classes: {}".format(y_test.shape[-1]))

(35000, 32, 32, 3) (35000, 10)
N classes: 10


##### Model

In [3]:
tf.debugging.set_log_device_placement(True)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    sub_model = tf.keras.applications.VGG19(include_top=False, input_shape=(32, 32, 3), classes=10)
    sub_model.trainable = False

    flat = tf.keras.layers.Flatten()(sub_model.layers[-1].output)
    classify_ = tf.keras.layers.Dense(10, activation='softmax')(flat)
    model = tf.keras.Model(inputs=sub_model.inputs, outputs=classify_)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.05), loss='categorical_crossentropy', metrics=['accuracy'])
    
    lr_decay = tf.keras.callbacks.ReduceLROnPlateau()
    check_pt = tf.keras.callbacks.ModelCheckpoint('./cache/best_param.hdf5', monitor='val_loss', save_best_only=True)

    EPOCHS = 500
    BATCH_SIZE = 300
    CB = [lr_decay, check_pt]


    history  = model.fit(x=x_train, 
                          y=y_train,
                          epochs=EPOCHS,
                          validation_data=(x_val,  y_val),
                          batch_size=BATCH_SIZE,
                          callbacks=CB,
                          verbose=False)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/

Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op LogicalNot in device 

Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op LogicalNot in device /job:localhost/re

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op RandomUniform i

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op Fill in device 

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in 

KeyboardInterrupt: 

In [4]:
sub_model = tf.keras.applications.VGG19(include_top=False, input_shape=(32, 32, 3), classes=10)
sub_model.trainable = False

flat = tf.keras.layers.Flatten()(sub_model.layers[-1].output)
classify_ = tf.keras.layers.Dense(10, activation='softmax')(flat)
model = tf.keras.Model(inputs=sub_model.inputs, outputs=classify_)


model.load_weights('./cache/best_param.hdf5')

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/r

Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/rep

In [21]:
model.layers[]

SyntaxError: invalid syntax (<ipython-input-21-a6df008e4b42>, line 1)

In [19]:
from utils.serialization import save_pickle
layer_names = [layer.name for layer in model.layers]
save_pickle(layer_names, './cache/layer_seq.pickle')

##### Model Performance

In [ ]:
from utils.numpy import to_onehot
from sklearn.metrics import multilabel_confusion_matrix

y_hat = model.predict(x_test)
y_label = to_onehot(np.argmax(y_hat, axis=1), 10)


cm = multilabel_confusion_matrix(y_test, y_label)

##### Runtime

In [ ]:
def layer_runtime(index, batch, model, repeat=150, return_size=False):
    '''
    Parameters
    ----------
    index: int:
        layer index
    model: keras.model
    repeate: int
    
    Return
    ------
    runtime: [sec, sec, sec...] 
    dsize: /kbyte
    '''
    
    
    front_model = tf.keras.Model(model.input, model.layers[index-1].output)
    front_output = front_model(batch)
    
    runtimes = []
    for _ in range(repeat):
        start_time = timeit.default_timer()
        model.layers[index](front_output)
        end_time = timeit.default_timer()
        
        runtime = end_time - start_time
        runtimes.append(runtime)
    
    if return_size == False:
        return runtimes
    else:
        return_val = model.layers[index](front_output)
        mem_size = return_val.numpy().nbytes / 1024
        return runtimes, mem_size


resource_consump = dict()

for i in range(0, len(model.layers)):
    runtimes, d_size = layer_runtime(i, x_train[0:1], model, repeat=50, return_size=True)
    
    l_name = model.layers[i].name
    resource_consump[l_name] = {'runtime':runtimes, 'dsize':d_size}

In [ ]:
df = []

for layer_name in list(resource_consump.keys()):
    _df = pd.DataFrame(resource_consump[layer_name]['runtime'])
    _df['layer_name'] = layer_name
    df.append(_df)
    
runtimes = pd.concat(df)
runtimes.columns = ['runtime', 'layer_name']
runtimes.head(2)

In [ ]:
df = []
for layer_name in list(resource_consump.keys()):
    _dsize = resource_consump[layer_name]['dsize']
    df.append([layer_name, _dsize])
    
d_size = pd.DataFrame(df)
d_size.columns = ['layer_name', 'dsize']
d_size.head(2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(figsize=(10 ,10))
sns.boxplot(y='runtime', x='layer_name', data=runtimes, orient='v')


plt.ylim(-0.0001, 0.001)
plt.xticks(rotation=80)

ax2 = plt.twinx()
sns.lineplot(x='layer_name', y='dsize', data=d_size, ax=ax2, markers='*', color='red', linestyle='--')

In [ ]:
# Procedure

config = {'exit_point'}
def get_exitpoint(config, bandwidth, input_data_batch):
    '''
    config
    '''